In [1]:
import numpy as np
import sympy as sp

In [2]:
# The look up table takes the form list[\sigma, ...] + list[Excitations]
# lookup_table = {
# 0: [],  # | c > b_0 b_0^\dagger b_0 b_0^\dagger
# 1: [0, 1],  # | \Phi_1^2 > +
# 2: [0, 2],  # | \Phi_1^3 > +
# 3: [1, 1],  # | \Phi_1^2 > -
# 4: [1, 2],  # | \Phi_1^3 > -
# 5: [0, 1, 1, 1],  # | \Phi_{1, 1}^{2, 2} >
# 6: [0, 1, 1, 2],  # | \Phi_{1, 1}^{2, 3} >
# 7: [0, 1, 2, 1],  # | \Phi_{1, 1}^{3, 2} >
# 8: [0, 1, 2, 2],  # | \Phi_{1, 1}^{3, 3} >
# }

In [3]:
lookup_table = {
    0: [],
    1: [0, 1],  # +
    2: [0, 1],  # -
    3: [0, 2],  # +
    4: [0, 2],  # -
}

In [4]:
from utils import read_elements, Z
from energy import Energy


groundstate = np.array(
    [
        [1, 0, 0],
        [1, 0, 0],
    ]
)
energy = Energy(groundstate)
values = energy.values


def onebody(i: int) -> sp.Expr:
    return -(Z**2) / (2 * (i + 1) ** 2)


def ground_excited(i: int, a: int) -> sp.Expr:
    return values[i, 0, a, 0]


def excited_excited(i: int, a: int, j: int, b: int) -> sp.Expr:
    if i != j and a != b:
        # return values[a, j, i, b] / 2
        return -values[a, j, b, i] / 2
    elif i == j and a != b:
        # Have taken into account spin
        return values[a, j, b, j] / 2
    elif i != j and a == b:
        return 0
    else:
        a_term = onebody(a)
        i_term = onebody(i)
        onebody_term = a_term - i_term + energy.E0_ref
        interaction_term = values[a, 0, a, 0] / 2 + 3 * values[0, 0, 0, 0] / 2
        return onebody_term + interaction_term
        return (onebody(a) - onebody(i) + energy.E0_ref) + 1 / 2
        print(f"{a_term=}, {i_term=}, {energy.E_ref=}")
        return onebody(a) - onebody(i) + energy.E_ref


def get_matrix_elem(idx_bra: int, idx_ket: int) -> sp.Expr:
    bra = lookup_table[idx_bra]
    ket = lookup_table[idx_ket]

    if len(bra) == 0 and len(ket) == 0:
        return energy.E_ref
    if len(bra) == 0:
        return ground_excited(ket[0], ket[1])
    if len(ket) == 0:
        return ground_excited(bra[0], bra[1])
    if len(bra) == 2 and len(ket) == 2:
        return excited_excited(bra[0], bra[1], ket[0], ket[1])
    raise ValueError(f"Invalid bra {bra} and ket {ket}")

In [5]:
hamiltonian = sp.zeros(len(lookup_table), len(lookup_table))
for i, bra in lookup_table.items():
    for j, ket in lookup_table.items():
        hamiltonian[i, j] = get_matrix_elem(i, j)

In [6]:
def get_energy_of_hamiltonian(hamiltonian: sp.Matrix, z_value: int):
    inserted = hamiltonian.subs(Z, z_value)
    eigenvalues = [
        key for key, value in inserted.eigenvals().items() for _ in range(value)
    ]
    approx_eigenvalues = [sp.re(value.evalf()) for value in eigenvalues]
    energy = min(approx_eigenvalues)
    return energy


def to_eV(energy: float) -> float:
    return energy * 2 * 13.6

In [7]:
approx_energy = get_energy_of_hamiltonian(hamiltonian, 2)
print(f"Approximate energy: {approx_energy} atomic units")
print(f"Approximate energy: {to_eV(approx_energy)} eV")

Approximate energy: -2.75964255305295 atomic units
Approximate energy: -75.0622774430402 eV


In [8]:
hamiltonian.eigenvals()

{-121*Z**2/108 + 3108319*Z/1990656 - (-45*Z**4/4 + 246515599*Z**3/7962624 - 2603689829238897008556701677032472697*Z**2/130586304825673650646771680000000000 + (121*Z**2/36 - 3108319*Z/663552)**2)/(3*(75*Z**6/4 - 135030835*Z**5/1769472 + 400464642397422077342462375112776894339*Z**4/4178761754421556820696693760000000000 - 32112414194756553985513788061030263277*Z**3/928613723204790404599265280000000000 + (121*Z**2/36 - 3108319*Z/663552)**3 - (121*Z**2/4 - 3108319*Z/73728)*(15*Z**4/4 - 246515599*Z**3/23887872 + 2603689829238897008556701677032472697*Z**2/391758914477020951940315040000000000)/2 + sqrt(-4*(-45*Z**4/4 + 246515599*Z**3/7962624 - 2603689829238897008556701677032472697*Z**2/130586304825673650646771680000000000 + (121*Z**2/36 - 3108319*Z/663552)**2)**3 + (75*Z**6/2 - 135030835*Z**5/884736 + 400464642397422077342462375112776894339*Z**4/2089380877210778410348346880000000000 - 32112414194756553985513788061030263277*Z**3/464306861602395202299632640000000000 + 2*(121*Z**2/36 - 3108319*Z/

In [9]:
hamiltonian.subs(Z, 2).evalf()

Matrix([
[             -2.75, 0.0893550334194116, 0.0893550334194116, 0.0439594494960981, 0.0439594494960981],
[0.0893550334194116, -0.415123456790123, -0.415123456790123, 0.0505264769127863, 0.0505264769127863],
[0.0893550334194116, -0.415123456790123, -0.415123456790123, 0.0505264769127863, 0.0505264769127863],
[0.0439594494960981, 0.0505264769127863, 0.0505264769127863, -0.247734917534722, -0.247734917534722],
[0.0439594494960981, 0.0505264769127863, 0.0505264769127863, -0.247734917534722, -0.247734917534722]])

In [10]:
hamiltonian

Matrix([
[        -Z**2 + 5*Z/8,            2048*sqrt(2)*Z/64827,            2048*sqrt(2)*Z/64827,           1269*sqrt(3)*Z/100000,           1269*sqrt(3)*Z/100000],
[ 2048*sqrt(2)*Z/64827,         -5*Z**2/8 + 1351*Z/1296,         -5*Z**2/8 + 1351*Z/1296, 777959424*sqrt(6)*Z/75429903125, 777959424*sqrt(6)*Z/75429903125],
[ 2048*sqrt(2)*Z/64827,         -5*Z**2/8 + 1351*Z/1296,         -5*Z**2/8 + 1351*Z/1296, 777959424*sqrt(6)*Z/75429903125, 777959424*sqrt(6)*Z/75429903125],
[1269*sqrt(3)*Z/100000, 777959424*sqrt(6)*Z/75429903125, 777959424*sqrt(6)*Z/75429903125,       -5*Z**2/9 + 16175*Z/16384,       -5*Z**2/9 + 16175*Z/16384],
[1269*sqrt(3)*Z/100000, 777959424*sqrt(6)*Z/75429903125, 777959424*sqrt(6)*Z/75429903125,       -5*Z**2/9 + 16175*Z/16384,       -5*Z**2/9 + 16175*Z/16384]])